In [2]:
# Import dependencies
import pandas as pd
import json
import pprint
import sys
import urllib
from bs4 import BeautifulSoup as bs
import time
import requests
import pymongo
from splinter import Browser
import json
import numpy as np
import plotly.express as px
# from config import yelp_key

In [ ]:
# Set up the basics for calling the Yelp API
# https://python.gotrained.com/yelp-fusion-api-tutorial/
# Cities gathered from https://en.wikipedia.org/wiki/List_of_municipalities_in_Missouri
yelp_key = "xOIRKCHjHkBJKYhXXIFtROFa8FVcVAYB63jdcF-DqiR5BT16M67ZR0gYqlwtiCWZLCG0Rh1xoeTb0FAdJFiEllZI463MC3M8DwfENshTGKmfWDVWoi-2binUNC3DXnYx"
api_key= yelp_key
headers = {'Authorization':'Bearer %s' % api_key}
url='https://api.yelp.com/v3/businesses/search'
offset = [0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900]
cities = ["Kansas City MO","St. Louis MO","Springfield MO","Independence MO",
          "Columbia MO","Lee's Summit MO","O'Fallon MO","St. Joseph MO",
          "St. Charles MO","Blue Springs MO","St. Peters MO","Florissant MO",
          "Joplin MO","Chesterfield MO","Jefferson City MO","Cape Girardeau MO",
          "Oakville MO","Wildwood MO","University City MO","Ballwin MO","Raytown MO",
          "Liberty MO","Wentzville MO","Mehlville MO","Kirkwood MO","Maryland Heights MO",
          "Hazelwood MO","Gladstone MO","Grandview MO","Belton MO","Webster Groves MO",
          "Sedalia MO","Ferguson MO","Arnold MO","Affton MO","Spanish Lake MO",
          "Rolla MO","Raymore MO","Old Jamestown MO","Nixa MO","Warrensburg MO",
          "Manchester MO","Hannibal MO","Creve Coeur MO","Ozark MO","Kirksville MO",
          "Poplar Bluff MO","Lemay MO","Concord MO","Sikeston MO","Farmington MO",
          "Overland MO","Clayton MO","Fort Leonard Wood MO","Republican MO",
          "Jennings MO","Lake St. Louis MO","Lebanon MO","Carthage MO",
          "Washington MO","Moberly MO","Jackson MO","Marshall MO",
          "St. Ann MO","Grain Valley MO","Fulton MO","West Plains MO",
          "Maryville MO","Crestwood MO","Neosho MO","Festus MO",
          "Bridgeton MO","Mexico MO","Dardenne Prairie MO",
          "Excelsior Springs MO","Webb City MO","Kennett MO",
          "Bellefontaine Neighbors MO","Town and Country MO",
          "Troy MO","Branson MO","Bolivar MO","Union MO","Eureka MO",
          "Harrisonville MO","Cameron MO","Chillicothe MO","Ellisville MO","Clinton MO",
          "Berkeley MO","Monett MO","Park Hills MO","Murphy MO","Richmond Heights MO",
          "Ladue MO","Sunset Hills MO","Smithville MO","Nevada MO","Kearney MO",
          "Des Peres MO","Boonville MO","Perryville MO","Pleasant Hill MO",
          "Brentwood MO","Maplewood MO","Warrenton MO","Dexter MO","Oak Grove MO",
          "Olivette MO","Sappington MO","Aurora MO","Carl Junction MO","Sullivan MO",
          "Pacific MO","Valley Park MO","Black Jack MO","Bonne Terre MO","Marshfield MO",
          "St. John MO","De Soto MO","Shrewsbury MO","Caruthersville MO","Trenton MO",
          "Charleston MO","Glendale MO","Richmond MO","Barnhart MO","Battlefield MO",
          "Parkville MO","Pevely MO","Macon MO","Weldon Spring MO","Glasgow Village MO",
          "Bowling Green MO","Odessa MO","Willard MO","Greenwood MO","Savannah MO",
          "Desloge MO","Dellwood MO","Normandy MO","Salem MO","Crystal City MO",
          "Waynesville MO","Higginsville MO","Mountain Grove MO","Breckenridge Hills MO",
          "Lexington MO","St. Clair MO","Imperial MO","Platte City MO","Rock Hill MO",
          "Peculiar MO","Mount Vernon MO","Eldon MO","Scott City MO","Brookfield MO",
          "Lamar MO","Hollister MO","Ste. Genevieve MO","Osage Beach MO","St. Robert MO",
          "High Ridge MO","California MO","Malden MO","Northwoods MO","Butler MO",
          "St. James MO","North Kansas City MO","Woodson Terrace MO","Centralia MO",
          "Fenton MO","Fredericktown MO","Castle Point MO","Vandalia MO","Carrollton MO",
          "Camdenton MO","Ashland MO","Palmyra MO","El Dorado Springs MO","Frontenac MO",
          "Herculaneum MO","Louisiana MO","Cuba MO","Sugar Creek MO","Pagedale MO",
          "Bethany MO","Pine Lawn MO","Cassville MO","Tipton MO","Holts Summit MO",
          "Portageville MO","East Prairie MO","Licking MO","Wright City MO",
          "New Madrid MO","Buffalo MO","Buckner MO","Cottleville MO","Rogersville MO",
          "Ava MO","Pleasant Valley MO","Chaffee MO","Hayti MO","Riverside MO",
          "Windsor MO","Riverview MO","Montgomery City MO","Hillsboro MO","Byrnes Mill MO",
          "Bel-Ridge MO","Mountain View MO","Knob Noster MO","Gray Summit MO","Fayette MO",
          "Owensville MO","Potosi MO","Villa Ridge MO","Clarkson Valley MO",
          "Green Park MO","Whiteman AFB MO","Monroe MO","Moscow Mills MO","Versailles MO",
          "Lawson MO","Concordia MO","Country Club MO","Moline Acres MO","Hermann MO",
          "Kimberling City MO","Oronogo MO","Canton MO","Strafford MO","Seneca MO",
          "Terre du Lac MO","Plattsburg MO","Wellston MO","Forsyth MO","Holden MO",
          "Thayer MO","Marceline MO","Marionville MO","Village of Four Seasons MO",
          "Willow Springs MO","Marlborough MO","Steele MO","Cabool MO","Clever MO",
          "Granby MO","Warsaw MO","Hanley Hills MO","New Haven MO","Lathrop MO",
          "Houston MO","Kahoka MO","Doniphan MO","Campbell MO","Piedmont MO",
          "Warson Woods MO","Anderson MO","Milan MO","Bernie MO","Lake Lotawana MO",
          "Elsberry MO","Bloomfield MO","Seymour MO","Carterville MO","Vinita Park MO",
          "Unionville MO","Richland MO","Slater MO","Noel MO","St. Paul MO","Memphis MO",
          "Stockton MO","Hamilton MO","Gallatin MO","Senath MO","Duquesne MO",
          "Merriam Woods MO","Sparta MO","Albany MO","Weatherby Lake MO",
          "Cedar Hill MO","Shelbina MO","Adrian MO","Garden City MO","Steelville MO",
          "Weston MO","Bourbon MO","Salisbury MO","Lake Ozark MO","Tarkio MO",
          "Huntsville MO","LaBarque Creek MO","Dixon MO","Winchester MO","Bismarck MO",
          "Belle MO","Gower MO","Wardsville MO","Bel-Nor MO","Hallsville MO",
          "Sweet Springs MO","Hillsdale MO","Marble Hill MO","Ash Grove MO","Crane MO",
          "Ironton MO","Linn MO","Lakeshire MO","Claycomo MO","Velda City MO",
          "Winfield MO","Rich Hill MO","Fair Grove MO","Oakland MO","Shell Knob MO",
          "Greenfield MO","La Plata MO","Charlack MO","Advance MO","Gerald MO",
          "St. George MO","Winona MO","Sarcoxie MO","Rock Port MO","Mansfield MO",
          "Oran MO","Calverton Park MO","Pierce City MO","Clarkton MO","Leadwood MO",
          "Country Club Hills MO","Goodman MO","Paris MO","Wellsville MO",
          "Cool Valley MO","Lilbourn MO","Lincoln MO","Stanberry MO","Edina MO",
          "Archie MO","Princeton MO","Mound City MO","La Monte MO","Marthasville MO",
          "Lake Winnebago MO","Appleton City MO","Cole Camp MO","Duenweg MO",
          "Maysville MO","Crocker MO","Kissee Mills MO","Glasgow MO","Purdy MO",
          "Stover MO","Gideon MO","New Franklin MO","Velda Village Hills MO",
          "Lone Jack MO","Humansville MO","Billings MO","King City MO"]
offset2 = [0,50,100]

In [ ]:
# Set up empty lists to prepare for the API call
restaurant_name = []
restaurant_lat = []
restaurant_lon = []
restaurant_address = []
restaurant_state = []
loop_check = []
response_number = 0

In [ ]:
# Call the API - Put N/A instead of failing
for y in cities:
    response_number = 0
    for x in offset2:
        params = {'location':[y],'price':'1','term':'fast food','limit':50,'offset':[x]}
        try:
            req=requests.get(url, params=params, headers=headers)
            response = req.json()
            for x in response["businesses"]:
                #print(response["businesses"][response_number]["name"])
                #print(response["businesses"][response_number]["coordinates"]["latitude"])
                #print(response["businesses"][response_number]["coordinates"]["longitude"])
                try:
                    restaurant_name.append(response["businesses"][response_number]["name"])
                except:
                    restaurant_name.append("N/A")

                try:
                    restaurant_lat.append(response["businesses"][response_number]["coordinates"]["latitude"])
                except:
                    restaurant_lat.append("N/A")

                try:
                    restaurant_lon.append(response["businesses"][response_number]["coordinates"]["longitude"])
                except:
                    restaurant_lon.append("N/A")

                try:
                    restaurant_address.append(response["businesses"][response_number]["location"]["display_address"])
                except:
                    restaurant_address.append("N/A")

                try:
                    restaurant_state.append(response["businesses"][response_number]["location"]["state"])
                    loop_check.append([y])
                except:
                    restaurant_state.append("N/A")
                    loop_check.append([y])

                if response_number == 49:
                    response_number = 0
                else: 
                    response_number = response_number + 1
        except:
            restaurant_name.append("N/A")
            restaurant_lat.append("N/A")
            restaurant_lon.append("N/A")
            restaurant_address.append("N/A")
            restaurant_state.append("N/A")

In [ ]:
# Load into a dataframe
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
d = {'name':restaurant_name, 'latitude':restaurant_lat,'longitude':restaurant_lon,'address':restaurant_address,'state':restaurant_state,'loopcheck':loop_check}
restaurant_df = pd.DataFrame(data=d)

In [ ]:
# Save to CSV
restaurant_df.to_csv("../Resources/restaurant_data.csv",index=True,header=True)

In [13]:
# Filter down to unique
cleaned_restaurant_df = pd.read_csv("../Resources/restaurant_data.csv")
cleaned_restaurant_df = cleaned_restaurant_df.drop_duplicates(subset= "address", keep = "first")

In [14]:
cleaned_restaurant_df['address'] = cleaned_restaurant_df['address'].str.strip('[]')
cleaned_restaurant_df['loopcheck'] = cleaned_restaurant_df['loopcheck'].str.strip('[]')
cleaned_restaurant_df['address'] = cleaned_restaurant_df['address'].str.strip("'")
cleaned_restaurant_df['address'] = cleaned_restaurant_df['address'].str.replace(r"[\"\',]", '')
cleaned_restaurant_df['loopcheck'] = cleaned_restaurant_df['loopcheck'].str.replace(r"[\"\',]", '')
cleaned_restaurant_df = cleaned_restaurant_df.reset_index(drop=True)
cleaned_restaurant_df = cleaned_restaurant_df.drop(columns=['Unnamed: 0'])
cleaned_restaurant_df = cleaned_restaurant_df.loc[cleaned_restaurant_df["state"]=="MO",:]
cleaned_restaurant_df = cleaned_restaurant_df.reset_index(drop=True)
cleaned_restaurant_df.to_csv("../Resources/cleaned_restaurant_data.csv", index = True, header = True)


In [5]:
poverty_file='../Resources/poverty.csv'
poverty_df=pd.read_csv(poverty_file)
narrowed_poverty_df=poverty_df[['Year', 'State', 'County ID', 'State / County Name','All Ages in Poverty Count', 'All Ages in Poverty Percent']]


In [6]:
unemp_file="../Resources/unemployment.csv"
unemp_df=pd.read_csv(unemp_file)
narrowed_unemp=unemp_df[['County ID','Labor Force','Employed','Unemployed','Unemployment Rate(%)']]


In [4]:
combined_df=narrowed_poverty_df.merge(narrowed_unemp,how='outer', on='County ID')
mo=combined_df[combined_df['State']==29]
mo=mo.reset_index(drop=True)
mo=mo.rename(columns={'Unemployment Rate(%)':'UnemploymentRate', 'County ID':'CountyID', 'State / County Name':'County',
                      'All Ages in Poverty Count':'AllAgesinPovertyCount','All Ages in Poverty Percent':'AllAgesinPovertyPercent',
                     'Labor Force':'LaborForce'})
mo_data='../Resources/mo_data.csv'
mo.to_csv(mo_data)



NameError: name 'narrowed_poverty_df' is not defined

In [16]:
from scipy import stats 
mo_data='../Resources/mo_data.csv'
mo=pd.read_csv(mo_data)
pov=mo['AllAgesinPovertyCount']
unemp=mo['Unemployed']
stats.ttest_ind(pov, unemp) 

Ttest_indResult(statistic=28.746327526153404, pvalue=4.3066809637914987e-78)